# 1 Read me

Here I am going to show the code I made for the control system and also the steps I took in getting the result. For the micro-controller I had the [LAUNCHXL-F28027F](https://www.ti.com/tool/LAUNCHXL-F28027F?ds_k=LAUNCHXL-F28027F&DCM=yes&gclsrc=ds&gclsrc=ds). The boost converter was already made and was not designed for optimal closed-loop implementation, but more on that later.  

# 2 Method of implementing controls

I have 2 options if I want to implement the control program. Option 1 I design the entire thing in Simulink and then use the built in Embedded Coder Simulink submodel and then let it do all the data conversion. This way I only need to use Simulink and then press built hardware and I am done. The second option is I make the entire code in C using the free Code Composer Studio program from Texas Instruments.

My C/C++ skill is very little, plus it can become really easy to be lost when designing and more importantly debugging problems if I decide to use this option. Simulink is a lot more user friendly, easier to follow and debug, but it isn't free sadly. In practice controls engineers do complicated controls using Simulink/Matlab, thanks to my university I also get both for free so I decided to do everything in Matlab/Simulink. Maybe later I can try and do this is C/C++, but I can't promise this.

# 3 Simulated model

The first step before implementing any control algorithm is to first make a simulation model of the thing u want to control. Since I am going to implement everything in Simulink/Matlab I decided to make a Simulink/Matlab simulation model. Below is a reminder of how the Boost converter looks like

<img src="Images/Boost_circuit_with_input_cap.png" width=500> 

In my special case I have full control over the input voltage, so I decided to use $45 V$, because that was the one the university used and I couldn't find any reason to change that. I could  keep swap all passive components to get the new desired output values, but that is a pretty dumb idea for obvious reasons. In this case I have control over the load, but in practice this is not always the case. So what can I control? There is only one thing left the transistors gate signal. I can control how fast it turn ON/OFF and how long it does that. And that is what I will use

## Open loop model

I am going to start with open loop control simulation model. Purely because it is the easiest to implement. 

<img src="Images/General_open_loop_control.png" width=500> 
[Source](https://www.electronicshub.org/open-loop-system/)

In my case the input signal is the switching frequency and duty cycle and the process output is the output voltage of the boost converter.

The model i made is shown below. It can be found in [Fake_Boost]{Fake_Boost}. Yes I ignored all parasitics for my model, but I just wanted a simple implementation to see if what I think happens actually happens. I didn't want to go measure and derive a perfect simulation model that will exactly follow the measured result.

<img src="Images/Fake_boost_open_loop_method_model.png" width=500> 

<img src="Images/Fake_boost_open_loop_method_model_plot.png" width=500> 

Um the MOSFET model looks right but the IGBT overshoots a lot... Something is wrong.

For full transparency I am also adding the Maltab code I used to to upload all the parameters. It can be found in the Fake_boost_parameters Matlab code 

```Matlab
clc; clear; close all;

%% General Parameters
V_DC=45; % 
R_Load=400; 
C_out=25e-3;
V_f=1.5; 

%% MOSFET 
V_MOSFET_out=150;
L_MOSFET=750e-6;
D_MOSFET=(V_MOSFET_out+V_f-V_DC)/(V_MOSFET_out+V_f)*100; % Calculated duty cycle
t_sw_MOSFET=1/(80e3);

%% IGBT
V_IGBT_out=300; 
L_IGBT=1e-3;
D_IGBT=(V_IGBT_out+V_f-V_DC)/(V_IGBT_out+V_f)*100; % Calculated duty cycle
t_sw_IGBT=1/(15e3);
```

<img src="Images/Fake_boost_open_loop_maskes_transistors.png" width=500> 

So my first idea is that it is the IGBT model, since everything else looks fine. So that is what I did I swapped the IGBT MASk for a MOSFET mask

<img src="Images/Fake_boost_open_loop_method_model_plot_both_MOSFET.png" width=500> 

So I can swap the IGBT for a MOSFET to get a realistic result, but that makes no sense since when doing the actual measurements the output voltage does not overshoot, else it's overvoltage protection would kick in. So there has to be something stopping this voltage overshoot with the IGBT model? Oh wait the parasitic resistance of the inductor that i ignored... 

I now added the $R_{DC}$ to the inductor and hopefully that gives me my expected result.

<img src="Images/Fake_boost_open_loop_method_model_solution.png" width=500> 

<img src="Images/Fake_boost_open_loop_method_model_plot_solution.png" width=700> 

```Matlab
clc; clear; close all;

%% General Parameters
V_DC=45; % 
R_Load=400; 
C_out=25e-3;
V_f=1.5; 

%% MOSFET 
V_MOSFET_out=150;
L_MOSFET=750e-6;
D_MOSFET=(V_MOSFET_out+V_f-V_DC)/(V_MOSFET_out+V_f)*100; % Calculated duty cycle
t_sw_MOSFET=1/(80e3);
R_DC_MOSFET=0.3;

%% IGBT
V_IGBT_out=300; 
L_IGBT=1e-3;
D_IGBT=(V_IGBT_out+V_f-V_DC)/(V_IGBT_out+V_f)*100; % Calculated duty cycle
t_sw_IGBT=1/(15e3);
R_DC_IGBT=0.1;

```

Lesson learned don't always ignore parasitics...

## Closed  loop model

Open loop works pretty well, however as was noticed the output voltage is not exactly the desired output voltage. That is because the controller doesn't know that they are different. It only sends a fix duty cycle as was pre-calculated. So to fix this problem a closed loop system is implemented.

<img src="Images/General_open_loop_control.png" width=500> 
[Source](https://electronicscoach.com/closed-loop-control-system.html)

There are 2 methods I found current and voltage feedback, but the designed only has voltage feedback for over-voltage protection. Altium messed up and the short circuit current protection wasn't implemented correctly.. So I am only going to do voltage feedback:p 